In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
# base_df = pd.read_csv("../no_repeated.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
base_df = pd.read_csv("../BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",usecols=['SessionID', 'TIMESTAMP', 'URL_FILE', ])
base_df.rename(columns={'URL_FILE':'Activity'}, inplace=True)
base_df["TIMESTAMP"] = pd.to_datetime(base_df["TIMESTAMP"], infer_datetime_format=True)
base_df = base_df.sort_values(["SessionID", "TIMESTAMP"])
base_df.head()

,SessionID,TIMESTAMP,Activity
3273278,46,2015-11-06 08:07:22.780,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken
5333642,46,2015-11-06 08:07:40.767,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...
3733243,46,2015-11-06 08:07:51.390,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...
5904405,46,2015-11-06 08:08:06.003,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...
5573282,46,2015-11-06 08:08:19.343,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...


In [3]:
# Helper functions
def create_df_for_pivot(df: pd.DataFrame):
  df = df.copy()
  #Create consecutive column with the consecutive activity and count how many time each pair occurs
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

  #Count all the pairs and safe in new DataFrame
  df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

  df_for_pivot_1.fillna("end_session", inplace=True)
  return df_for_pivot_1
  
def make_pivot(df, index_names, column_names):
  df = df.copy()
  """
  Functions that takes in a DataFrame and returns a pivot table with all the chances
  
  Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
  """
  df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
  
  #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
  df_chances["total_row_count"] = df_chances.sum(axis=1)
  df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
  
  df_chances.fillna(0, inplace=True)
  df_chances.drop("total_row_count", axis=1, inplace=True)

  return df_chances

  
# Create df for pivot
df_for_pivot = create_df_for_pivot(base_df)
df_chances_1 = make_pivot(df_for_pivot, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = base_df.groupby("SessionID").nth(0)["Activity"].value_counts() / base_df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

df_chances_total = df_chances_1 
df_chances_total

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
final_df = df_chances_total.iloc[:, :-1]
# final_df = remove_circles(final_df)
final_df

level_1,/,//,//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,//werk_nl/werknemer/contact,//werk_nl/werknemer/contact/email_uw_vraag,//werk_nl/werknemer/contact/vestiging-zoeken,//werk_nl/werknemer/home,//werk_nl/werknemer/over-werk-nl/handleiding,//werk_nl/werknemer/over-werk-nl/handleiding/cv-plaatsen,//werk_nl/werknemer/over-werk-nl/handleiding/sollicitatie-wijziging,...,/xpsitem/wdo_013389,/xpsitem/wdo_013407,/xpsitem/wdo_013827,/xpsitem/wdo_014041,/zoeken_portlet/ajax/addVacatureToGereageerd,/zoeken_portlet/ajax/zoekAantalIndicatief,/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet_wg/ajax/selectContactpersoon,/zoeken_portlet_wg/ajax/selectVacature,end_session
level_0,,,,,,,,,,,,,,,,,,,,,
/,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.073834
//,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.035714
//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact/email_uw_vraag,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/zoeken_portlet/ajax/addVacatureToGereageerd,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017588,0.002513,0.005025,0.0,0.0,0.055276
/zoeken_portlet/ajax/zoekAantalIndicatief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000107,0.537184,0.051269,0.0,0.0,0.007539
/zoeken_portlet/ajax/zoekBeroep,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.441592,0.489910,0.0,0.0,0.003979


In [4]:
patterns = pd.read_csv("./gen_patterns/Swapped.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
patterns.sort_values("Activity_0_x", inplace=True)

# change column names
patterns_df = patterns.rename(columns={'Activity_0_x':'A','Activity_1_x':'B'})
patterns_df
grouped = patterns_df.drop(["count_x"	,"count_y",	"dif","fac"],axis=1).groupby("A")['B'].apply(list)
group_dict = grouped.to_dict()
# group_dict

In [11]:
def swapping(_df:pd.DataFrame,event_1:str,event_2:str,deviation_rate):
  df = _df.copy()
  res = []
  deviation_rate = deviation_rate / 100
  for i,val in df.loc[:,event_1].items():
    if val > 0 :
      old_val = df.loc[i,event_1]
      new_val = old_val * (1 - deviation_rate)
      df.loc[i,event_1] = new_val

      df.loc[i,event_2] += old_val - df.loc[i,event_1]
      row = {'from':i,'to':event_2,'inject':event_1,}
      res = res + [row]
        
  
  return (df,res)

In [6]:
def swap_multiple_events(dic,deviation_rate,_matrix):
  res = []
  matrix = _matrix.copy()
  for key, value in dic.items():
    len_list = len(value)
    dev = deviation_rate / len_list
    for i in range(len_list):
      mat,row = swapping(matrix,key,value[i],dev)
      matrix = mat
      res.append(row)
    
  final = []
  for i in range(len(res)):
    for j in range(len(res[i])):
      final.append(res[i][j])
  return final,matrix
    

In [12]:
swapped_res,swapped_matrix = swap_multiple_events(group_dict,10,final_df)

In [23]:
swapped_res_df = pd.DataFrame(swapped_res)
swapped_res_df


,from,to,inject
0,//werk_nl/werknemer/werkmap,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,//werk_nl/werknemer/werkmap
1,/portal/page/portal/werk_nl/werknemer/sollicit...,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,//werk_nl/werknemer/werkmap
2,/werk_nl/werknemer/home,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,//werk_nl/werknemer/werkmap
3,/werk_nl/werknemer/mijn_werkmap,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,//werk_nl/werknemer/werkmap
4,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,//werk_nl/werknemer/werkmap
...,...,...,...
4147,/xdocs/ux-frontend/fonts/uwvsanlig-webfont.woff,/shared/foutopgetreden.html,/zoeken_portlet/ajax/zoekAantalIndicatief
4148,/xpsimage/WDO212444,/shared/foutopgetreden.html,/zoeken_portlet/ajax/zoekAantalIndicatief
4149,/zoeken_portlet/ajax/addVacatureToGereageerd,/shared/foutopgetreden.html,/zoeken_portlet/ajax/zoekAantalIndicatief
4150,/zoeken_portlet/ajax/zoekAantalIndicatief,/shared/foutopgetreden.html,/zoeken_portlet/ajax/zoekAantalIndicatief


In [43]:
def gen_markov_chain(amount:int,start_chances,_matrix:pd.DataFrame, swapped_res_df:pd.DataFrame) -> list:
  matrix = _matrix.copy()
  generated_sessions = []
  while len(generated_sessions) < amount:
    flag = False
    new_session = []
    # choose start activity
    start_activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
    new_session.append(start_activity[0])
    
    while new_session[-1] != 'end_session':
      curr_activity = new_session[-1]
      row_activity = matrix.loc[curr_activity,:]
      new_activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values))[0]
      new_session.append(new_activity)
      if ((swapped_res_df['from'] == curr_activity) & (swapped_res_df['to'] == new_activity)).any():
        # pattern is generated
        # we need to generate a new activity from new activity but first inject the inject activity
        new_activity_patt = np.random.choice(a = list(matrix.loc[new_activity,:].keys()), size = 1, p = list(matrix.loc[new_activity,:].values))[0]
        inject_activity = swapped_res_df.loc[(swapped_res_df['from'] == curr_activity) & (swapped_res_df['to'] == new_activity),'inject'].values[0]
        new_session.append(inject_activity)
        new_session.append(new_activity_patt)
        flag = True
    
    if flag: 
      generated_sessions.append(new_session)
      if len(generated_sessions)  % 50 == 0 : print(len(generated_sessions))

  return generated_sessions

In [44]:
test = gen_markov_chain(100,start_chances,swapped_matrix,swapped_res_df)

50
100


In [45]:
def markov_result_to_df(markov_result, name_index, name_csv):
    activity_list_final = []
    session_id_list = []
    for nbr in range(len(markov_result)):
        string_generated = name_index + str(nbr)
        for i in markov_result[nbr]:
            if i != "end_session":
                activity_list_final.append(i)
                session_id_list.append(string_generated)
                
    df_generated = pd.DataFrame(list(zip(session_id_list, activity_list_final)),
               columns =['SessionID', 'URL_FILE'])
    df_generated.to_csv(name_csv,index=False)

In [46]:
deviation_rates = [5,10,25,50,75,100]

In [47]:
def gen_all_sessions(rates:list[int],amount:int,base_df=final_df, start_chances_df=start_chances, swapped_res_df=swapped_res_df):
  for rate in rates:
    print(f'start with rate: {rate}')
    print(f'Generate the prob matrix for rate: {rate}')
    new_df = base_df.copy()
    swapped_res,swapped_matrix = swap_multiple_events(group_dict,rate,new_df)
    print(f'Generate sessions rate: {rate}')
    sessions = gen_markov_chain(amount,start_chances_df,swapped_matrix,swapped_res_df)
    print(f'Sessions Generated')
    markov_result_to_df(sessions,f'rate_{rate}_',f'gen_sessions/{rate}_{amount}.csv')
    print(f'done with {rate}')

In [48]:
gen_all_sessions(deviation_rates,1000)

start with rate: 5
Generate the prob matrix for rate: 5
Generate sessions rate: 5
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 5
start with rate: 10
Generate the prob matrix for rate: 10
Generate sessions rate: 10
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 10
start with rate: 25
Generate the prob matrix for rate: 25
Generate sessions rate: 25
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 25
start with rate: 50
Generate the prob matrix for rate: 50
Generate sessions rate: 50
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 50
start with rate: 75
Generate the prob matrix for rate: 75
Generate sessions rate: 75
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 75
start with rate: 10